In [2]:
## This notebook implements the computations for "(Non-)vanishing results for extensions between simple functors on free groups"

## The cells below contain some instructions about how to use these algorigthms. For more information, please refer to 
# the README on the GitHub repository: https://github.com/louishainaut/Ext-Outer-Functors.

## For any questions or comments, you can find my current email address on my personal webpage https://louishainaut.github.io/ .

## The first cell contains all the definitions, it only needs to be run once at the beginning.
# Unless your goal is to proofread the algorithm or expand it, you will not need to understand the contents of this first cell

s = SymmetricFunctions(QQ).schur()

# This function returns the Ext group corresponding to 'source' and 'target'. These parameters can each be either
# a non-negative integer, or an integer partition.
def Access_ExtGroup(Ext, source, target):
    if (source in ZZ) and (target in ZZ):
        if (source > target) or (source < 0):
            raise ValueError(f"Error when calling 'Access_ExtGroup'. When the parameters 'source' and 'target' are both integers, they must be non-negative and 'source' must be at most as large as 'target'. Given values: 'source' : {source} and 'target' : {target}")
        if target >= len(Ext):
            raise ValueError(f"Error when calling 'Access_ExtGroup'. The parameter 'target' : {target} must be smaller than the length of 'Ext' : {len(Ext)}.")
        return Ext[target][source]
    
    if (source in ZZ) and (target in Partitions()):
        m = sum(target)
        if (source < 0) or (source > m):
            raise ValueError(f"Error when calling 'Access_ExtGroup'. The parameters 'source' : {source} and 'target' : {target} are not in a valid range.")
        if (m >= len(Ext)):
            raise ValueError(f"Error when calling 'Access_ExtGroup'. The parameter 'target' : {target} must be a partition of an integer smaller than the length of 'Ext' : {len(Ext)}.")
        return sum(val*s(key[0]) for key, val in Ext[m][source] if key[1] == target)
    
    if (source in Partitions()) and (target in ZZ):
        n = sum(source)
        if (n < 0) or n > target:
            raise ValueError(f"Error when calling 'Access_ExtGroup'. The parameters 'source' : {source} and 'target' : {target} are not in a valid range.")
        if target >= len(Ext):
            raise ValueError(f"Error when calling 'Access_ExtGroup'. The parameter 'target' : {target} must be smaller than the length of 'Ext': {len(Ext)}.")
        return sum(val*s(key[1]) for key, val in Ext[target][n] if key[0] == source)
    
    if (source in Partitions()) and (target in Partitions()):
        n = sum(source)
        m = sum(target)
        source = Partition(source)
        target = Partition(target)
        if (n < 0) or (n > m):
            raise ValueError(f"Error when calling 'Access_ExtGroup'. The parameters 'source' : {source} and 'target' : {target} are not in a valid range.")
        if (m >= len(Ext)):
            raise ValueError(f"Error when calling 'Access_ExtGroup'. The parameter 'target' : {target} must be a partition of an integer smaller than the length of 'Ext' : {len(Ext)}.")
        return Ext[m][n].monomial_coefficients().get((source, target), 0)
    
    raise ValueError(f"Error when calling 'Access_ExtGroup'. The parameters 'source' : {source} and 'target' : {target} can only be a non-negative integer or an integer partition.")

def Find_NonKoszul(Ext):
    result = []
    for m, line in enumerate(Ext):
        for n, el in enumerate(line):
            for key, val in el:
                if (-1)^(m-n)*val < 0:
                    result.append((key, val))
                    
    return result
    
def Compute_ExtGroups(bound, Outer = True, CompFact = None, check = True):
    if check and CompFact is not None:
        Verify_CompFact(bound, CompFact)
        
    if check and Outer and (bound > 10):
        raise ValueError(f"Error when calling 'Compute_ExtGroups' with the parameter 'Outer' set to True (default value) and 'bound' having value {bound}: any value of 'bound' larger than 10 gives incomplete results. Also please be aware that large values of 'bound' become computationally intensive. Please set 'bound' to a value no larger than 10, or add the parameter 'check = False' if you know what you are doing.")
        
    if CompFact is None:
        CompFact = All_Composition_Factors(bound, Outer)
    
    result = [[s.one().tensor(s.one())]]
    for n in range(1, bound+1):
        new_line = [s.zero().tensor(s.zero())]*n
        for i in range(n):
            for smaller in Partitions(i):
                for larger in Partitions(n):
                    CC = Compute_Ext_CC(smaller, larger, result, CompFact)
                    new_line[i] += (-sum(CC))*s(smaller).tensor(s(larger))
        new_line.append(sum(s(part).tensor(s(part)) for part in Partitions(n)))
        result.append(new_line)
        
    return result

# Implements the formula from Remark 4.20, and return the computed value, as well as the difference with the actual value
def Compare_ExtGroups(ExtOut, ExtAll, diff, n):
    Test = sum(s[0].tensor(s[i])*ExtAll[n+diff-i][n] for i in range(diff+1)) - s[n].tensor(s[n+diff])
    if (diff%2 == 0) and (n>=2):
        Test -= s[n].omega().tensor(s[n+diff].omega())
        
    return (Test, ExtOut[n+diff][n] - Test)

def Compute_Ext_CC(smaller, larger, Ext, CompFact):
    m = sum(larger)
    n = sum(smaller)
    return [sum(CompFact[m][p].monomial_coefficients().get((larger, middle), 0)*
                Ext[p][n].monomial_coefficients().get((smaller, middle), 0)
                for middle in Partitions(p)) for p in range(n,m)]

def Verify_CompFact(bound, CompFact):
    if len(CompFact) != bound + 1:
        raise ValueError(f"The parameter 'CompFact' has {len(CompFact)} terms, while it is supposed to have length one more than the value of the parameter 'bound', which is {bound}.")
    for i, el in enumerate(CompFact):
        if len(el) != i+1:
            raise ValueError(f"The element of 'CompFact' in position {i} should be a least of length {i+1}, but instead it has length {len(el)}.")
    return #no return value

# Auxiliary computations for the composition factors

#Compute all composition factors up to (and including) 'bound'. You should not need to call any of the functions below that one.
# The parameter 'Outer' is set to True by default, in which case the composition factors or \omega\beta S_\lambda are computed.
# If 'Outer' is set to false, then the composition factors of \beta S_\lambda are computed instead, as they are those needed
# to compute the Ext groups studied by Vespa.
# You can set the parameter 'Partial' to True if you only want to get the lower bound guaranteed by theoretical arguments. This
# option is not recommended unless you know what you are doing. The parameter 'Partial' has no effect if 'Outer' is set to False.
def All_Composition_Factors(bound, Outer = True, Partial = False):
    result = [[s.one().tensor(s.one())]]
    if Outer:
        for n in range(1, bound+1):
            Hom = Homology_ConfSpace(n, Partial = Partial)
            result.append([s.zero().tensor(s.zero()) + sum(val*s(key[0]).omega().tensor(s(key[1])) for key, val in SymFunc) 
                           for SymFunc in Hom[1]])
        return result
    else:
        CTot = [[s.zero().tensor(s.zero())]*(bound + 1), #The value of this first element does not matter here, but an element has to be present
                [Lie(i).degree_negation().omega().tensor(s[1]) for i in range(bound+1)]]

        for n in range(1, bound+1):
            result.append([s.zero().tensor(s.zero()) + sum(val*s(key[0]).omega().tensor(s(key[1])) 
                                                           for key, val in E1(n, line, n-line, CTot))
                              for line in range(n+1)])
        return result

#The following function returns the homology of configuration space of n points on a wedge of circles
#The parameter n should be a positive integer not larger than 10. Values larger than 10 are accepted, but you will not get
# the correct answer.
#If you set the parameter Partial to True, you will not obtain the full homology, but only a lower bound guaranteed to
# be correct from theoretical arguments
def Homology_ConfSpace(n, Partial = False):
    Hom = Compute_LowerBound(n)
    Hom = Improve_LowerBound(n, Hom, Partial = Partial)
    return Hom

# returns the decomposition of the term in arity n of the Lie operad into irreducibles
p = SymmetricFunctions(QQ).power()
def Lie(n):
    if n == 0:
        return s.zero()
        
    result = sum(moebius(d)*p[d]^Integer(n/d) for d in divisors(n))
    return s(result/n)

# This function and the next one implement by hand the plethysm of s[n] with SLie \otimes H(X), so that only the
# necessary terms are computed and are added to the appropriate term on the E1 page
def aux_func(part, chain_comp):
    l = part.to_exp()
    return prod( s[l[i]].plethysm(chain_comp[i+1]) for i in range(len(l)) if l[i] != 0)


def E1(particles, line, col, CTot):
    result = s.zero().tensor(s.zero())
    for k in range(particles-line-col, particles-line+1):
        for part1 in Partitions(particles-k, length = line):
            result += aux_func(part1, CTot[1])*sum(aux_func(part2, CTot[0]) 
                                                   for part2 in Partitions(k, length = particles-line-col))

    return (-1)^(line+col)*result

# Computations for the E2-page

# Takes the alternating sum of a line and distributes the terms in the two possibly non-zero terms according to
# their sign (would provide the correct answer if there were no cancellation in the Euler characteristic)
def Compute_LowerBound(particles):
    if (particles not in ZZ) or (particles < 1):
        raise ValueError("Cannot call function 'Compute_LowerBound' with the given parameter 'particles'. Only positive integer values are accepted.")
    bound = particles+1

    # Returns the tensor product of SLie with the cohomology of X, graded by the degree of the cohomology part
    # The sign is chosen according to the degree of the corresponding term in the tensor product, so that Koszul
    # duality is automatically implemented
    CTot = [[-Lie(i).degree_negation().omega().tensor(s.one()) for i in range(bound)],
            [Lie(i).degree_negation().omega().tensor(s[1]) for i in range(bound)]]

    if particles == 1:
        return[[E1(particles, 0, 0, CTot)], [s.zero().tensor(s.zero()), E1(particles, 1, 0, CTot)]] # Since F(X, 1) = X
    
    #else:
    Hom = [[s.zero().tensor(s.zero())],[s.zero().tensor(s.zero())]]

    for line in range(1, particles):
        alt_sum = sum((-1)^(line+col+particles)*E1(particles, line, col, CTot) 
                      for col in range(particles-line+1))
        Hom[0].append(alt_sum.map_coefficients(lambda coeff : max(0, -coeff)))
        Hom[1].append(alt_sum.map_coefficients(lambda coeff : max(0, coeff)))
    
    Hom[1].append(E1(particles, particles, 0, CTot))
    return Hom

# When Partial is set to True it only corrects the multiplicities whose value follows from a theoretical argument
# (i.e. the symmetric and exterior powers).
# When Partial is set to False it additionnally corrects the multiplicities according to specific computations
# performed in joint work of Gadish and Hainaut
def Improve_LowerBound(particles, Hom, Partial = False):
   # Finds the correct multiplicity for the symmetric and exterior powers
    for n in range(1, particles):
        error = s.zero() + Focus_GL(Hom[0][n-1], [n-1]) - Focus_GL(Hom[1][n], [n])
        Hom[0][n] += error.tensor(s[n])
        Hom[1][n] += error.tensor(s[n])

    for n in range(2, particles):
        error = s.zero() + Focus_GL(Hom[0][n-2], [1]*(n-2)) - Focus_GL(Hom[1][n], [1]*n)
        Hom[0][n] += error.tensor(s[n].omega())
        Hom[1][n] += error.tensor(s[n].omega())

    if Partial:
        return Hom

    #else:
    # Adds corrections to the remaining Schur functors if there are no more than 10 particles; incomplete corrections for 11 particles
    if particles <= 6:
        return Hom #no correction needed
    
    if particles == 7:
        error = s[4,2,1].tensor(s[2,1])
        Hom[0][3] += error
        Hom[1][3] += error

    if particles == 8:
        error = (2*s[5,2,1] + s[4,3,1] + s[4,2,2] + s[4,2,1,1] + s[3,3,1,1] + s[3,2,2,1]).tensor(s[2,1])
        Hom[0][3] += error
        Hom[1][3] += error
        error = s[6,2].tensor(s[2,1,1])
        Hom[0][4] += error
        Hom[1][4] += error

    if particles == 9:
        error = (s[6,3] + 2*s[6,2,1] + s[6,1,1,1] + 2*s[5,3,1] + s[5,2,2] + 3*s[5,2,1,1] + 2*s[4,4,1] + 3*s[4,3,2] + 4*s[4,3,1,1] + 4*s[4,2,2,1] + 3*s[4,2,1,1,1] + 2*s[3,3,2,1] + s[3,3,1,1,1] + s[3,2,2,2] + 2*s[3,2,2,1,1]).tensor(s[2,1])
        Hom[0][3] += error
        Hom[1][3] += error
        error = (s[7,2] + s[5,2,2] + s[4,4,1] + s[3,2,2,2]).tensor(s[3,1])
        error += (s[6,2,1] + s[5,3,1]).tensor(s[2,1,1])
        Hom[0][4] += error
        Hom[1][4] += error
        error = s[7,2].tensor(s[2,1,1,1])
        Hom[0][5] += error
        Hom[1][5] += error

    if particles == 10: 
        #Schur functor [2,1]
        error = s[7,3] + 2*s[7,2,1] + s[7,1,1,1] + s[6,4] + 5*s[6,3,1] + 2*s[6,2,2] + 5*s[6,2,1,1] + s[6,1,1,1,1] + 4*s[5,4,1] + 6*s[5,3,2] + 9*s[5,3,1,1] + 7*s[5,2,2,1] + 6*s[5,2,1,1,1] + s[5,1,1,1,1,1] + 3*s[4,4,2] + 4*s[4,4,1,1] + 4*s[4,3,3] + 10*s[4,3,2,1] + 7*s[4,3,1,1,1] + 4*s[4,2,2,2] + 9*s[4,2,2,1,1] + 3*s[4,2,1,1,1,1] + 2*s[3,3,3,1] + 3*s[3,3,2,2] + 6*s[3,3,2,1,1] + 2*s[3,3,1,1,1,1] + 3*s[3,2,2,2,1] + 3*s[3,2,2,1,1,1] + s[2,2,2,2,1,1]
        Hom[0][3] += error.tensor(s[2,1])
        Hom[1][3] += error.tensor(s[2,1])
        #Schur functor [3,1]
        error = s[7,2,1] + 2*s[6,3,1] + s[6,2,1,1] + s[5,4,1] + s[5,3,2] + 2*s[5,3,1,1] + s[5,2,2,1] + s[4,4,2] + s[4,3,3] + 2*s[4,3,2,1] + s[4,2,2,2] + s[4,2,2,1,1] + s[3,3,2,1,1] + s[3,2,2,2,1]
        Hom[0][4] += error.tensor(s[3,1])
        Hom[1][4] += error.tensor(s[3,1])
        #Schur functor [2,2]
        error = s[4,4,2]
        Hom[0][4] += error.tensor(s[2,2])
        Hom[1][4] += error.tensor(s[2,2])
        #Schur functor [2,1,1]
        error = s[8,1,1] + s[7,3] + s[7,2,1] + s[7,1,1,1] + 2*s[6,3,1] + 2*s[6,2,1,1] + s[5,4,1] + s[5,3,2] + s[5,3,1,1] + s[5,2,2,1] + s[4,4,1,1] + s[4,3,3]
        Hom[0][4] += error.tensor(s[2,1,1])
        Hom[1][4] += error.tensor(s[2,1,1])
        #Schur functor [3,1,1]
        error = s[8,2]
        Hom[0][5] += error.tensor(s[3,1,1])
        Hom[1][5] += error.tensor(s[3,1,1])
        #Schur functor [2,1,1,1]
        error = s[7,2,1] + s[6,3,1]
        Hom[0][5] += error.tensor(s[4,1].omega())
        Hom[1][5] += error.tensor(s[4,1].omega())
        #Schur functor [2,1^4]
        error = s[8,2]
        Hom[0][6] += error.tensor(s[5,1].omega())
        Hom[1][6] += error.tensor(s[5,1].omega())

    if particles == 11:

        #Schur functor [2,1]
        error = s[8,3] + 3*s[8,2,1] + s[8,1,1,1] + s[7,4] + 7*s[7,3,1] + 3*s[7,2,2] + 7*s[7,2,1,1] + s[7,1,1,1,1]
        error += 2*s[6,5] + 7*s[6,4,1] + 11*s[6,3,2] + 14*s[6,3,1,1] + 12*s[6,2,2,1] + 11*s[6,2,1,1,1] + 3*s[6,1,1,1,1,1]
        error += 5*s[5,5,1] + 10*s[5,4,2] + 13*s[5,4,1,1] + 9*s[5,3,3] + 26*s[5,3,2,1] + 15*s[5,3,1,1,1] + 7*s[5,2,2,2]
        error += 19*s[5,2,2,1,1] + 10*s[5,2,1,1,1,1] + s[5,1,1,1,1,1,1] + 4*s[4,4,3] + 15*s[4,4,2,1] + 10*s[4,4,1,1,1]
        error += 13*s[4,3,3,1] + 14*s[4,3,2,2] + 26*s[4,3,2,1,1] + 12*s[4,3,1,1,1,1] + 12*s[4,2,2,2,1] + 14*s[4,2,2,1,1,1]
        error += 6*s[4,2,1,1,1,1,1] + 6*s[3,3,3,2] + 5*s[3,3,3,1,1] + 12*s[3,3,2,2,1] + 11*s[3,3,2,1,1,1] + 2*s[3,3,1,1,1,1,1]
        error += 3*s[3,2,2,2,2] + 7*s[3,2,2,2,1,1] + 5*s[7,3,1].omega() + s[6,5].omega() + s[7,4].omega()
        Hom[0][3] += error.tensor(s[2,1])
        Hom[1][3] += error.tensor(s[2,1])
        #Schur functor [3,1]
        error = s[9,1,1] + s[8,3] + s[8,2,1] + s[8,1,1,1] + 4*s[7,3,1] + 4*s[7,2,1,1] +s[6,5] + 3*s[6,4,1] + 5*s[6,3,2]
        error += 5*s[6,3,1,1] + 4*s[6,2,2,1] + 3*s[6,2,1,1,1] + 2*s[5,5,1] + 3*s[5,4,2] + 6*s[5,4,1,1] + 6*s[5,3,3]
        error += 8*s[5,3,2,1] + 3*s[5,3,1,1,1] + s[5,2,2,2] + 6*s[5,2,2,1,1] + s[4,4,3] + 5*s[4,4,2,1] + 3*s[4,4,1,1,1]
        error += 4*s[4,3,3,1] + 5*s[4,3,2,2] + 6*s[4,3,2,1,1] + s[4,3,1,1,1,1] + 3*s[4,2,2,2,1] + 2*s[4,2,2,1,1,1]
        error += 3*s[3,3,3,2] + 3*s[3,3,2,2,1] + 2*s[3,3,2,1,1,1] + 2*s[3,2,2,2,1,1] + s[6,5].omega()
        Hom[0][4] += error.tensor(s[3,1])
        Hom[1][4] += error.tensor(s[3,1])
        #Schur functor [2,2]
        error = s[7,2,1,1] + s[6,4,1] + s[6,3,2] + s[5,4,2] + s[5,4,1,1] +s[5,3,3] + s[5,3,2,1] + s[4,4,2,1] + s[4,3,3,1] + s[4,3,2,2]
        Hom[0][4] += error.tensor(s[2,2])
        Hom[1][4] += error.tensor(s[2,2])
        #Schur functor [4,1]
        error = s[9,2]
        Hom[0][5] += error.tensor(s[4,1])
        Hom[1][5] += error.tensor(s[4,1])

    return Hom

#Helper functions, you should not need to call them directly

def Focus_GL(element, part):
    return sum(coeff*s[index[0]]*(index[1]==part) for index, coeff in element)

def ExpForm(part, Latex = False):
    if part == []:
        return "(0)"
    mult = part.to_exp()
    result = "("
    for i,exp in reversed([el for el in enumerate(mult)]):
        if exp == 1:
            result += f"{i+1},"
        if exp > 1:
            if Latex:
                result += f"{i+1}^{'{'}{exp}{'}'},"
            else:
                result += f"{i+1}^{exp},"
                
    return result[:-1]+")"

def Print_Scalar(val, Latex = False):
    if val == 1:
        return ""
    
    if val < 0:
        if Latex:
            return f"({val})\\cdot"
        else:
            return f"({val})*"
    
    if Latex:
        return f"{val}\\cdot"
    else:
        return f"{val}*"

def Latex_Form(SymFunc):
    if SymFunc == 0:
        return "0"
    
    result = ""
    decomp = sorted(SymFunc.monomial_coefficients().items())
    
    key, val = decomp[0]
    result += Print_Scalar(val, True) + f"(\\Specht{'{'}{ExpForm(key[0], Latex = True)}{'}'}\\boxtimes \\Specht{'{'}{ExpForm(key[1], Latex = True)}{'}'})"
    
    for key, val in decomp[1:]:
        result += f" + " + Print_Scalar(val, True) + f"(\\Specht{'{'}{ExpForm(key[0], Latex = True)}{'}'}\\boxtimes \\Specht{'{'}{ExpForm(key[1], Latex = True)}{'}'})"
#    if val == 1:
#        result += f"(\\Specht{'{'}{ExpForm(key[0])}{'}'}\\boxtimes \\Specht{'{'}{ExpForm(key[1])}{'}'})"
#    else:
#        result += f"{val}(\\Specht{'{'}{ExpForm(key[0])}{'}'}\\boxtimes \\Specht{'{'}{ExpForm(key[1])}{'}'})"
#        
#    for key, val in decomp[1:]:
#        if val == 1:
#            result += f" + (\\Specht{'{'}{ExpForm(key[0])}{'}'}\\boxtimes \\Specht{'{'}{ExpForm(key[1])}{'}'})"
#        else:
#            result += f" + {val}(\\Specht{'{'}{ExpForm(key[0])}{'}'}\\boxtimes \\Specht{'{'}{ExpForm(key[1])}{'}'})"
            
    return result

def Text_Form(SymFunc):
    if SymFunc == 0:
        return "0"
    
    result = ""
    decomp = sorted(SymFunc.monomial_coefficients().items())
    key, val = decomp[0]
    if val == 1:
        result += f"{ExpForm(key[0])}#{ExpForm(key[1])}"
    else:
        result += f"{val}*{ExpForm(key[0])}#{ExpForm(key[1])}"
        
    for key, val in decomp[1:]:
        if val == 1:
            result += f" + {ExpForm(key[0])}#{ExpForm(key[1])}"
        else:
            result += f" + {val}*{ExpForm(key[0])}#{ExpForm(key[1])}"
            
    return result

def Compute_Rank(SymFunc):
    if SymFunc == 0:
        return 0
    
    return sum(val*s(key[0]).scalar(s[1]^(sum(key[0])))*s(key[1]).scalar(s[1]^(sum(key[1]))) 
               for key,val in SymFunc.monomial_coefficients().items())

In [2]:
bound = 10
CompFact = All_Composition_Factors(bound)
Ext = Compute_ExtGroups(bound, CompFact = CompFact)

print(Text_Form(Access_ExtGroup(Ext, 4, 9)))
#print(Compute_Rank(Access_ExtGroup(Ext, 5, 7)))
#print(Access_ExtGroup(Ext, [1,1,1], [3,3,2,2]))
#print(Find_NonKoszul(Ext))

-1*(1^4)#(1^9) + -2*(1^4)#(2,1^7) + -3*(1^4)#(2^2,1^5) + -2*(1^4)#(2^3,1^3) + -1*(1^4)#(2^4,1) + -1*(1^4)#(3,1^6) + -1*(1^4)#(3,2,1^4) + -1*(1^4)#(3,2^2,1^2) + -1*(1^4)#(3,2^3) + -1*(1^4)#(3^2,1^3) + -4*(2,1^2)#(1^9) + -6*(2,1^2)#(2,1^7) + -8*(2,1^2)#(2^2,1^5) + -6*(2,1^2)#(2^3,1^3) + -4*(2,1^2)#(2^4,1) + -2*(2,1^2)#(3,1^6) + -5*(2,1^2)#(3,2,1^4) + -2*(2,1^2)#(3,2^2,1^2) + -2*(2,1^2)#(3,2^3) + -2*(2,1^2)#(3^2,1^3) + -1*(2,1^2)#(3^2,2,1) + -1*(2^2)#(1^9) + -4*(2^2)#(2,1^7) + -4*(2^2)#(2^2,1^5) + -4*(2^2)#(2^3,1^3) + -2*(2^2)#(2^4,1) + -3*(2^2)#(3,1^6) + -3*(2^2)#(3,2,1^4) + -3*(2^2)#(3,2^2,1^2) + -1*(2^2)#(3^2,2,1) + -2*(3,1)#(1^9) + -6*(3,1)#(2,1^7) + -7*(3,1)#(2^2,1^5) + -6*(3,1)#(2^3,1^3) + -3*(3,1)#(2^4,1) + -4*(3,1)#(3,1^6) + -4*(3,1)#(3,2,1^4) + -4*(3,1)#(3,2^2,1^2) + -2*(3,1)#(3,2^3) + -2*(3,1)#(3^2,1^3) + -1*(3,1)#(3^2,2,1) + -1*(3,1)#(4^2,1) + (4)#(1^9) + -2*(4)#(2,1^7) + -1*(4)#(2^2,1^5) + -2*(4)#(2^3,1^3) + -1*(4)#(2^4,1) + -3*(4)#(3,1^6) + -1*(4)#(3,2,1^4) + -2*(4)#(3,2^2,1^

In [21]:
bound = 10
#ExtOut = Compute_ExtGroups(bound, Outer = True)
ExtOut = Ext #If Ext has not already been computed, replace this line with the previous one
ExtAll = Compute_ExtGroups(bound, Outer = False)

n = 3
diff = 3 # the sum n+diff should not exceed 'bound'
computed, error = Compare_ExtGroups(ExtOut, ExtAll, diff, n)
print(computed)
print(error)

-s[1, 1, 1] # s[1, 1, 1, 1, 1, 1] - s[1, 1, 1] # s[2, 2, 1, 1] - s[2, 1] # s[1, 1, 1, 1, 1, 1] - 2*s[2, 1] # s[2, 1, 1, 1, 1] - s[2, 1] # s[2, 2, 1, 1] - s[3] # s[1, 1, 1, 1, 1, 1] - s[3] # s[2, 1, 1, 1, 1] - s[3] # s[2, 2, 2]
-s[2, 1] # s[2, 2, 2]
